# 2 - Speech transcriptions

In [ ]:
%%capture
!pip install git+https://github.com/openai/whisper.git -q
!pip install stable-ts

In [ ]:
import whisper
from google.colab import drive
from pathlib import Path
import json
from tqdm.notebook import tqdm
import warnings

warnings.simplefilter("ignore", UserWarning)

from stable_whisper import stabilize_timestamps, load_model

In [ ]:
drive.mount('/content/drive')

songs_path = Path("drive/MyDrive/Máster/DLASP/Final/songs_separated/htdemucs")

Mounted at /content/drive


In [ ]:
#@title Auxiliar functions
def save_results(segments, text, folder):

  # Save text transcription
  with open(folder / "transcription.txt", "w") as f:
    f.write(text.strip())

  # Save segments timestamps
  for r in segments:
    if "tokens" in r:
      del r["tokens"]
    if "unstable_word_timestamps":
      del r["unstable_word_timestamps"]

  with open(folder / "transcription.json", "w") as fp:
      json.dump(segments, fp)

In [ ]:
model = load_model("large")

In [ ]:
for folder in (pbar := tqdm(list(songs_path.iterdir()), leave=False)):
  pbar.set_description(folder.name)

  audio = whisper.load_audio(folder / "vocals.mp3")
  results = model.transcribe(audio, task='transcribe', 
                             language='es', suppress_silence=True)
  stab_segments = stabilize_timestamps(results, top_focus=True)
  save_results(stab_segments, results["text"], folder)